# 作業目標:¶
利用手寫辨識資料說明MLP 中, 神經網路層的多寡跟預測結果的差異

# 作業重點:
[練習]請嘗試新增一層或兩層看看最後預測結果是否有差異
model.add(Dense(units=256, input_dim=784, kernel_initializer='normal', activation='relu'))


# 資料預處理

In [1]:
import numpy as np
import pandas as pd  

from keras.utils import np_utils

import matplotlib.pyplot as plt
#from keras.utils import plot_model
#from IPython.display import Image
    
np.random.seed(10)

Using TensorFlow backend.


In [2]:
#載入手寫辨識的資料集
from keras.datasets import mnist

( x_train_image, y_train_label ), ( x_test_image, y_test_label ) = mnist.load_data( )

In [3]:
# 訓練集及測試集資料的維度

print( 'Shape of Train : ',  x_train_image.shape )
print( 'Shape of Train Label : ',  y_train_label.shape )  
print( 'Shape of Test : ',  x_test_image.shape )  
print( 'Shape of Test Label : ',  y_test_label.shape )

Shape of Train :  (60000, 28, 28)
Shape of Train Label :  (60000,)
Shape of Test :  (10000, 28, 28)
Shape of Test Label :  (10000,)


In [4]:
#指定測試集與訓練資料集
x_Train = x_train_image.reshape( 60000, 784 ).astype( 'float32' )  # 784 = 28 * 28
x_Test =x_test_image.reshape( 10000, 784 ).astype( 'float32' )

print( 'Shape of Train : ',  x_Train.shape )
print( 'Shape of Train : ',  x_Test.shape )

Shape of Train :  (60000, 784)
Shape of Train :  (10000, 784)


In [5]:
# normalize inputs from 0-255 to 0-1
x_Train_normalize = x_Train / 255
x_Test_normalize = x_Test / 255

In [6]:
# OneHot Encoding : 把LABEL 轉成 NUMERICAL Categorical 
y_Train_OneHot = np_utils.to_categorical( y_train_label ) 
y_Test_OneHot = np_utils.to_categorical( y_test_label )

# 建立模型

In [7]:
from keras.models import Sequential
from keras.layers import Dense

In [8]:
#建立模型
model = Sequential( )

# 輸入層
model.add( Dense( units = 256,                        # 神經元數量
                         #input_dim = 784,  
                           input_shape = ( 784, ), 
                           kernel_initializer = 'normal',   # 初始化權重的方法
                           activation = 'relu', 
                           name = 'Input_layer'
                           ) )

# 隱藏層
model.add( Dense( 128, kernel_initializer = 'normal', activation = 'relu', name = 'Hidden_1' ) )    # 建立有 128 個神經元的隱藏層
model.add( Dense( 64, kernel_initializer = 'normal', activation = 'tanh', name = 'Hidden_2' ) )     # 建立有 64 個神經元的隱藏層

# 輸出層
model.add( Dense( units = 10,                           # 神經元數量
                           kernel_initializer = 'normal',    # 初始化權重的方法
                           activation = 'softmax', 
                           name = 'Output_Layer'
                           ) )


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
print( model.summary( ) )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (Dense)          (None, 256)               200960    
_________________________________________________________________
Hidden_1 (Dense)             (None, 128)               32896     
_________________________________________________________________
Hidden_2 (Dense)             (None, 64)                8256      
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# 模型網絡的流程圖  

#plot_model( model, to_file = 'Model.png' )
#Image( 'Model.png' )

# 訓練模型

In [11]:
model.compile( loss = 'categorical_crossentropy',  # 損失函數(Loss Function)
                      optimizer = 'adam',                      # 最佳化的演算法
                      metrics = [ 'accuracy' ]                # 模型成效的評量指標
                     )

In [12]:
train_history = model.fit( x = x_Train_normalize,
                                   y = y_Train_OneHot, 
                                   validation_split = 0.2,   # 驗證集佔訓練集的比例
                                   epochs = 10,               # 模擬次數
                                   batch_size = 20,          # 每批次的資料筆數
                                   verbose = 1                 # 顯示模型訓練進度 
                                   )

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 45s 934us/step - loss: 0.2467 - acc: 0.9284 - val_loss: 0.1147 - val_acc: 0.9658
Epoch 2/10
48000/48000 [==============================] - 41s 847us/step - loss: 0.0972 - acc: 0.9708 - val_loss: 0.1062 - val_acc: 0.9711
Epoch 3/10
48000/48000 [==============================] - 41s 846us/step - loss: 0.0678 - acc: 0.9791 - val_loss: 0.0960 - val_acc: 0.9726
Epoch 4/10
48000/48000 [==============================] - 41s 859us/step - loss: 0.0524 - acc: 0.9836 - val_loss: 0.0968 - val_acc: 0.9729
Epoch 5/10
48000/48000 [==============================] - 43s 896us/step - loss: 0.0409 - acc: 0.9870 - val_loss: 0.0857 - val_acc: 0.9753
Epoch 6/10
48000/48000 [==============================] - 42s 878us/step - loss: 0.0356 - acc: 0.9886 - val_loss: 0.0941 - val_acc: 0.9757
Epoch 7/10
48000/48000 [==============================] - 43s 893us/

# 以圖形顯示訓練過程

In [13]:
def show_train_history( train_history ) : 
        fig = plt.gcf( )
        fig.set_size_inches( 16, 6 )
        
        plt.subplot( 121 )
        plt.plot( train_history.history[ 'acc' ] )
        plt.plot( train_history.history[ 'val_acc' ] )
        plt.title( 'Train History' )
        plt.xlabel( 'Epoch' )
        plt.ylabel( 'Accuracy' )
        plt.legend( [ 'train', 'validation' ], loc = 'upper left' )
        
        plt.subplot( 122 )
        plt.plot( train_history.history[ 'loss' ] )
        plt.plot( train_history.history[ 'val_loss' ] )
        plt.title( 'Train History' )
        plt.xlabel( 'Epoch' )
        plt.ylabel( 'Loss' )
        plt.legend( [ 'train', 'validation' ], loc = 'upper left' )
        
        plt.show( )

In [14]:
# show_train_history( train_history )

# 評估模型準確率

In [15]:
scores = model.evaluate( x_Test_normalize, y_Test_OneHot )

print( 'accuracy = ', scores[1] )

10000/10000 [==============================] - 1s 76us/step
accuracy =  0.9746


# 測試集進行預測

In [16]:
prediction = model.predict_classes( x_Test_normalize ) 

In [17]:
# 混淆矩陣

pd.crosstab( y_test_label, prediction, rownames = [ 'label' ], colnames = [ 'predict' ] )

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,969,1,2,0,0,0,2,0,4,2
1,0,1116,2,0,0,1,3,0,12,1
2,0,0,1015,1,1,0,2,2,11,0
3,0,0,5,992,0,1,0,3,5,4
4,2,0,2,0,967,0,6,1,1,3
5,4,0,0,14,1,863,2,1,6,1
6,3,1,1,2,1,11,934,0,5,0
7,2,2,10,3,1,0,0,999,3,8
8,3,0,4,4,0,4,0,0,958,1
